In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-04-06 18:11:17--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2023-04-06 18:11:17 (7.62 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2023-04-06 18:11:17--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

In [ ]:
# Read data to datasets
train_data = pd.read_table(train_file_path,names=['labels','message'])
test_data = pd.read_table(test_file_path,names=['labels','message'])
test_data

,labels,message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


In [ ]:
MAX_LEN = len(test_data['message'].max())+1

In [ ]:
spam_text = str(test_data.iloc[1391][1])
spam_text


'not heard from u4 a while. call 4 rude chat private line 01223585334 to cum. wan 2c pics of me gettin shagged then text pix to 8552. 2end send stop 8552 sam xxx'

In [ ]:
train_data, train_labels = train_data.drop('labels',axis=1), train_data['labels']
test_data, test_labels = test_data.drop('labels',axis=1), test_data['labels']

In [ ]:
# Encoding Text to Int in Data

vocab = {}
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ")
  encoding = []

  for word in words:
    if word in vocab:
      code = vocab[word]
      encoding.append(code)
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1

  return encoding

train_data_real = []
test_data_real = []

for i in range(0,train_data.size):
  train_data_real.append(one_hot_encoding(train_data.iloc[i][0]))

for i in range(0,test_data.size):
  test_data_real.append(one_hot_encoding(test_data.iloc[i][0]))

In [ ]:
# Enconding Text to Int in Labels
train_labels_real = np.zeros(train_labels.size, dtype=np.int64)
for i in range(0,train_labels.size):
  if train_labels[i] == 'ham':
    train_labels_real[i] = 0
  elif train_labels[i] == 'spam':
    train_labels_real[i] = 1
train_labels_real


array([0, 0, 0, ..., 1, 1, 0])

In [ ]:
test_labels_real = np.zeros(test_labels.size, dtype=np.int64)
for i in range(0,test_labels.size):
  if test_labels[i] == 'ham':
    test_labels_real[i] = 0
  elif test_labels[i] == 'spam':
    test_labels_real[i] = 1
test_labels_real

array([0, 0, 0, ..., 0, 1, 1])

In [ ]:
from keras.preprocessing import sequence
from keras.utils import pad_sequences

In [ ]:
train_data_real = pad_sequences(train_data_real, MAX_LEN)
test_data_real = pad_sequences(test_data_real, MAX_LEN)

In [ ]:
VOCAB_SIZE = len(vocab)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 100, input_length=MAX_LEN),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

In [ ]:
history = model.fit(train_data_real, train_labels_real, epochs=10, validation_split=0.2)

Epoch 1/10
105/105 [==============================] - 30s 198ms/step - loss: 0.2532 - acc: 0.9133 - val_loss: 0.1048 - val_acc: 0.9737
Epoch 2/10
105/105 [==============================] - 7s 63ms/step - loss: 0.0838 - acc: 0.9767 - val_loss: 0.0727 - val_acc: 0.9821
Epoch 3/10
105/105 [==============================] - 4s 35ms/step - loss: 0.0481 - acc: 0.9880 - val_loss: 0.0503 - val_acc: 0.9892
Epoch 4/10
105/105 [==============================] - 2s 24ms/step - loss: 0.0315 - acc: 0.9928 - val_loss: 0.0510 - val_acc: 0.9868
Epoch 5/10
105/105 [==============================] - 4s 35ms/step - loss: 0.0187 - acc: 0.9967 - val_loss: 0.0526 - val_acc: 0.9856
Epoch 6/10
105/105 [==============================] - 2s 21ms/step - loss: 0.0115 - acc: 0.9973 - val_loss: 0.0580 - val_acc: 0.9856
Epoch 7/10
105/105 [==============================] - 1s 13ms/step - loss: 0.0065 - acc: 0.9988 - val_loss: 0.0726 - val_acc: 0.9821
Epoch 8/10
105/105 [==============================] - 1s 14ms/step 

In [ ]:
responses = model.evaluate(test_data_real,test_labels_real)
print('Accuracy: ' + str(round(responses[1],4)))

44/44 [==============================] - 0s 5ms/step - loss: 0.0862 - acc: 0.9749
Accuracy: 0.9749


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  class_dict = {
      0 : "ham",
      1 : "spam",
      }
  encoded_message = [one_hot_encoding(pred_text)]
  padded_message = pad_sequences(encoded_message, maxlen=MAX_LEN, padding='post')
  prediction = [model.predict(padded_message)[0][0], class_dict[np.round(model.predict(padded_message)[0][0])]]
  return prediction

pred_text = spam_text

prediction = predict_message(pred_text)
prediction

1/1 [==============================] - 0s 25ms/step


[0.9993185, 'spam']

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)

    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 19ms/step
You passed the challenge. Great job!
